In [7]:
# Import necessary libraries
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
import time
import pandas as pd

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


In [14]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager

# Function to initialize the WebDriver
def initialize_driver():
    options = webdriver.ChromeOptions()
    options.add_argument("--start-maximized")
    options.add_argument("--disable-blink-features=AutomationControlled")
    
    # Usar el argumento 'executable_path' en lugar de 'service'
    driver = webdriver.Chrome(executable_path=ChromeDriverManager().install(), options=options)
    return driver


In [15]:
# Function to scrape reviews from a Google Maps location
def scrape_google_maps_reviews(url, num_reviews=50):
    driver = initialize_driver()
    driver.get(url)
    
    # Wait for the reviews section to load
    time.sleep(5)
    
    # Scroll the page to load more reviews
    scrollable_div = driver.find_element(By.XPATH, '//div[contains(@class, "scrollable")]')
    
    for _ in range(num_reviews // 10):  # Modify this range depending on how many reviews to scrape
        driver.execute_script('arguments[0].scrollTop = arguments[0].scrollHeight', scrollable_div)
        time.sleep(2)
    
    # Extract review elements
    reviews = driver.find_elements(By.XPATH, '//div[contains(@class, "review")]')

    # Collecting review data
    reviews_data = []
    for review in reviews:
        review_text = review.find_element(By.XPATH, './/span[contains(@class, "review-text")]').text
        review_date = review.find_element(By.XPATH, './/span[contains(@class, "review-date")]').text
        reviews_data.append({"text": review_text, "date": review_date})
    
    driver.quit()
    
    return pd.DataFrame(reviews_data)

In [16]:
# Test the function with the Google Maps URL you provided
url = 'https://www.google.com/maps/place/Cervecer%C3%ADa+Sierra/@40.4355895,-3.7102454,17.36z/data=!4m6!3m5!1s0xd42285d9fe6f033:0xe27e4de86263432a!8m2!3d40.4342037!4d-3.7104844!16s%2Fg%2F11b6d43yfv?entry=ttu&g_ep=EgoyMDI0MDkxMS4wIKXMDSoASAFQAw%3D%3D'

# Scrape the reviews and store them in a DataFrame
reviews_df = scrape_google_maps_reviews(url, num_reviews=50)

# Display the first few reviews
reviews_df.head()

Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "/Users/jobandtalent/opt/anaconda3/envs/sar/lib/python3.9/site-packages/IPython/core/interactiveshell.py", line 3505, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/var/folders/hv/ckh3m6gn1sd45q2qctrqcwzh0000gn/T/ipykernel_64651/241914732.py", line 5, in <module>
    reviews_df = scrape_google_maps_reviews(url, num_reviews=50)
  File "/var/folders/hv/ckh3m6gn1sd45q2qctrqcwzh0000gn/T/ipykernel_64651/1974165106.py", line 10, in scrape_google_maps_reviews
    scrollable_div = driver.find_element(By.XPATH, '//div[contains(@class, "scrollable")]')
  File "/Users/jobandtalent/opt/anaconda3/envs/sar/lib/python3.9/site-packages/selenium/webdriver/remote/webdriver.py", line 976, in find_element
  File "/Users/jobandtalent/opt/anaconda3/envs/sar/lib/python3.9/site-packages/selenium/webdriver/remote/webdriver.py", line 321, in execute
    def _unwrap_value(self, value):
  File "/Users/jobandtalent/opt/anaconda3/envs/sar/lib/pyt